In [ ]:
!pip install pennylane nftopt

In [27]:
import numpy as np
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod
import time
import math

In [ ]:
class Experiment_Amplitude_embedding:
  def __int__(self,normalized_X,n_qubits,depth,weights,Case):
    self.state = normalized_X
    self.n_qubits = n_qubits
    self.depth = depth
    self.weights = weights
    self.Case = Case
    self.dev = qml.device("lightning.gpu",wires=self.n_qubits)

  # the functions we need
  def ansatz_layer_1(self,layer_weights):
    for dep in range(self.depth):
      for wire in range(self.n_qubits):
        qml.RY(layer_weights[wire+(self.n_qubits*dep)], wires=wire)

        for i in range(self.n_qubits-1):
            qml.CZ(wires=[i,i+1])
        qml.CZ(wires=[self.n_qubits-1,0])

  def ansatz_layer_2(self, layer_weights):
    for dep in range(self.depth):
        for wire in range(self.n_qubits):
            qml.RY(layer_weights[wire+(self.n_qubits*dep)], wires=wire)

        for i in range(self.n_qubits-1):
            qml.CZ(wires=[i,i+1])

    qml.Hadamard(0)

  @qml.qnode(dev)
  def circuit(self, weight):
    # since depth n was used
    if self.Case == 1:
      self.ansatz_layer_1(weight)
    if self.Case == 2:
      self.ansatz_layer_2(weight)
    # qml.state() applies Ua to Ini State, which gives Appro_a
    return qml.state()

  def variational_classifier(self, weights, x):
    # weights are thetas
    return np.real(self.circuit(weights))

  # Case I
  def F1_loss(self,a, Appro_a):
    F1 = 0.
    length = 2**self.n_qubits
    for i in range(length):
      F1 += (a[i]**2)*np.log2((Appro_a[i]**2)/(a[i]**2))

    #   Prepare the Bell state
    states = 2**self.n_qubits
    Bell_State=np.ones((states,1))/np.linalg.norm(np.ones((states,1)))
    #   The sum of vector a
    Sum_a=np.sum(a)
    sqrt_D = np.linalg.norm(np.ones((states,1)))
    UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F1)
    return np.real(UaFunction)

  # Case II
  def F2_loss(self,a,Appro_a):
    F2 = 0.
    k = []
    k_ind = []
    l = []
    l_ind = []
    # Split a into k and l, store indices as well(+ve and -ve)
    for i in range(len(a)):
      if(a[i] >= 0):
        k.append(a[i])
        k_ind.append(i)
      else:
        l.append(a[i])
        l_ind.append(i)

    for i in range(len(k)):
      F2 += (k[i]**2)*np.log2((Appro_a[k_ind[i]]**2)/(k[i]**2))
    D = 2 ** (self.n_qubits-1)
    for i in range(len(l)):
      F2 += (l[i]**2)*np.log2((Appro_a[l_ind[i]+D]**2)/(l[i]**2))


    #   Prepare the Bell state
    states_with_ancillary = 2**(self,n_qubits)
    Bell_State=np.ones((states_with_ancillary,1))/np.linalg.norm(np.ones((states_with_ancillary,1)))
    #   The sum of vector a
    Sum_a=np.sum(np.abs(a))
    sqrt_D = np.sqrt(2) * np.linalg.norm(np.ones((D,1)))
    UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F2)
    return np.real(UaFunction)

  cost_values = []
  # Case I
  def cost_1(self, weights, X):
    # X is our preparation state
    Appro_a = self.variational_classifier(weights, X)
    expectation_value = self.F1_loss(X, Appro_a)
    self.cost_values.append(expectation_value)
    return expectation_value

  # Case II
  def cost_2(self,weights, X):
    # X is our preparation state
    Appro_a = self.variational_classifier(weights, X)
    expectation_value = self.F2_loss(X, Appro_a)
    self.cost_values.append(expectation_value)
    return expectation_value

  def accuracy(labels, predictions):
    state0 = qml.math.dm_from_state_vector(labels)
    state1 = qml.math.dm_from_state_vector(predictions)
    return qml.math.fidelity(state0, state1)

In [134]:
class QHack:
  def __init__(self, state, depth):
    self.state = state
    self.depth = depth

    #state preparation
    state_0 = state
    # define the number of qubits we need
    self.n_qubits = int(math.log2(len(state_0)))
    # define weights
    self.weights = 4*np.pi*np.random.randint(self.n_qubits*self.depth)

    all_positive = all(n > 0 for n in x)
    all_negative = all(n < 0 for n in x)

    # Case I or II
    # Case I
    if all_positive or all_negative:

      # normalized
      summer = np.sum(state_0)
      normalized_X = state_0/summer

      temp=[]
      for i in range(len(normalized_X[0])):
        temp.append(np.sqrt(normalized_X[0][i]))
        x = np.array(temp)


      Q = Experiment_Amplitude_embedding(normalized_X,self.n_qubits,depth,self.weights,Case=1)
      result = optimize.minimize(cost_1, self.weights, method= nftmethod, options={'maxfev':2000})
      predictions = Q.variational_calssifier(result.x, x)
      self.acc = Q.accuracy(x,predictions)

    else:
      # Case II
      ## Rearranging the array so that there is no index confusion
      arr1 = []
      for i in range(len(state_0)):
        if(state_0[i] > 0):
          arr1.append(state_0[i])

      for i in range(len(state_0)):
        if(state_0[i] < 0):
          arr1.append(state_0[i])

      arr1 = np.array(arr1)

      # nromalized
      summer = np.sum(np.abs(arr1))
      normalized_X = arr1/summer

      temp = []
      for i in range(len(normalized_X)):
        val = np.sqrt(np.abs(normalized_X[i]))
        if (normalized_X[i] < 0):
          val = val * -1
        temp.append(val)
      x = np.array(temp)

      Q = Experiment_Amplitude_embedding(normalized_X,self.n_qubits,self.depth,self.weights,Case = 2)
      self.result = optimize.minimize(cost_2, self.weights, method= nftmethod, options={'maxfev':2000})
      predictions = Q.variational_calssifier(self.result.x, x)
      self.acc = Q.accuracy(x,predictions)


  # output params
  def params(self):
    return self.result.x

  # output cost function value
  #def fun():
    #return
  # output accuracy
  def acc(self):
    acc = self.acc
    return acc